### Military Operation

During a military operation, an officer must choose the members of a patrol of minimum size among the different soldiers who are under his command and whose skills are described in the table below:
| Soldier | Radio | Camouflage | Sniper | Physical | NCO |
| ------- | ----- | ---------- | ------ | -------- | --- |
| A       | 1     | 1          | 6      | 5        | 1   |
| B       | 0     | 1          | 8      | 7        | 0   |
| C       | 1     | 1          | 7      | 7        | 0   |
| D       | 1     | 0          | 4      | 9        | 0   |
| E       | 0     | 1          | 7      | 9        | 1   |
| F       | 1     | 0          | 8      | 6        | 0   |


The officer must respect the following conditions:
* One and only one non-commissioned officer is needed.
* At least 1 Radio operator and at least 1 Camouflage specialist are required.
* At least two radio operators will not be part of the patrol in order to remain available for other missions.
* Patrol members must have an average sniping rate of at least 7 and an average physical endurance rate of at least 7.5.
* If solder A is part of the patrol, then neither B nor C should be part of it.


\begin{align*}
\text{Decision variables:}\quad
& X_{A},X_{B},X_{C},X_{D},X_{E},X_{F}\in\{0,1\}\\[4pt]
\text{Objective function:}\quad
& \min Z= X_{A}+X_{B}+X_{C}+X_{D}+X_{E}+X_{F}\\[8pt]
\text{Subject to:}\\[4pt]
\text{NCO requirement: One and only one NCO is needed.}\;& X_{A}+X_{E}=1\\[2pt]
\text{Radio requirement: At least 1 Radio operator.}\;& X_{A}+X_{C}+X_{D}+X_{F}\ge 1\\[2pt]
\text{Radio requirement: At most two radio operators will be a part of the patrol.}\;& X_{A}+X_{C}+X_{D}+X_{F}\le 2\\[2pt]
\text{Camouflage requirement: At least 1 Camouflage specialist.}\;& X_{A}+X_{B}+X_{C}+X_{E}\ge 1\\[2pt]
\text{Sniping rate requirement: At least 7 of average sniping rate.}\;& 6X_{A}+8X_{B}+7X_{C}+4X_{D}+7X_{E}+8X_{F}\ge 7(X_{A}+X_{B}+X_{C}+X_{D}+X_{E}+X_{F})\\[2pt]
\text{Physical endurance rate requirement: At least 7.5 of average physical endurance}\;& 5X_{A}+7X_{B}+7X_{C}+9X_{D}+9X_{E}+6X_{F}\ge 7.5(X_{A}+X_{B}+X_{C}+X_{D}+X_{E}+X_{F})\\[2pt]
\text{Co-existence requirement: If solder A is part of the patrol, then neither B nor C should be part of it.}\;& 2X_{A}+X_{B}+X_{C}\le 2
\end{align*}

In [5]:
#import all the needed libraries
import gurobipy as gp
from gurobipy import GRB

In [6]:
data = { 'soldiers' : ['A', 'B', 'C', 'D', 'E', 'F'],
        'radio': {'A':1 , 'B':0, 'C':1, 'D':1, 'E':0, 'F':1},
        'camouflage': {'A':1 , 'B':1, 'C':1, 'D':0, 'E':1, 'F':0},
        'sniper': {'A':6, 'B':8, 'C':7, 'D':4, 'E':7, 'F':8},
        'physical': {'A':5, 'B':7, 'C':7, 'D':9, 'E':9, 'F':6},
        'NCO': {'A':1 , 'B':0, 'C':0, 'D':1, 'E':1, 'F':0}
        }

In [7]:
#build the model
model = gp.Model('military_operations')

#create the decision variables
X = model.addVars(data['soldiers'], vtype=GRB.BINARY, name='X')

In [8]:
#set the objective function
model.setObjective(sum(X[s] for s in data['soldiers']), GRB.MINIMIZE)

#add the constraints
model.addConstr(sum(data['NCO'][s] * X[s] for s in data['soldiers']) == 1, name='NCO_constraint')
model.addConstr(sum(data['radio'][s] * X[s] for s in data['soldiers']) >= 1, name='radio_in')
model.addConstr(sum(data['radio'][s] * X[s] for s in data['soldiers']) <= 2, name='radio_max')
model.addConstr(sum(data['camouflage'][s] * X[s] for s in data['soldiers']) >= 1, name='camouflage_in')
model.addConstr(sum(data['sniper'][s] * X[s] for s in data['soldiers']) >= 7 * X.sum(), name='avg_sniper')
model.addConstr(sum(data['physical'][s] * X[s] for s in data['soldiers']) >= 7.5 * X.sum(), name='avg_physical')
model.addConstr(2 * X['A'] + X['B'] + X['C'] <= 2, name='Coexist_A_B_C')

<gurobi.Constr *Awaiting Model Update*>

In [9]:
#turn off the Gurobi output
model.setParam('OutputFlag', 0)

#optimize the model
model.optimize()

#print the results
print(f"Total Soldiers Selected: {model.ObjVal}")
print('Selected Soldiers:', [s for s in data['soldiers'] if X[s].X > 0.5])

Total Soldiers Selected: 2.0
Selected Soldiers: ['E', 'F']
